In [2]:
import geopandas as gpd
import pandas as pd


In [ ]:

parcel_gdf = gpd.read_file("/Users/iansargent/Desktop/ORCA/Steamlit App Testing/VT_Parcel.geojson")


   OBJECTID           SPAN     GLIST_SPAN       MAPID      PARCID PROPTYPE  \
0         1  090-028-10516  090-028-10516  MIDDST-01R  MIDDST.01R   PARCEL   
1         2  090-028-10517  090-028-10517  00MTST-19L  00MTST.19L   PARCEL   
2         3  144-045-10159  144-045-10159      140105      140105   PARCEL   
3         4  144-045-10590  144-045-10590      442019      442019   PARCEL   
4         5  090-028-10518  090-028-10518  00MTST-20L  00MTST.20L   PARCEL   

   YEAR  GLYEAR      TOWN     TNAME  ... UVREDUC_HS UVREDUC_NR GLVAL_HS  \
0  2025  2024.0  BRIGHTON  Brighton  ...        0.0        0.0      0.0   
1  2025  2024.0  BRIGHTON  Brighton  ...        0.0        0.0      0.0   
2  2025  2024.0   CHESTER   Chester  ...        0.0        0.0   3158.0   
3  2025  2024.0   CHESTER   Chester  ...        0.0        0.0   1548.0   
4  2025  2024.0  BRIGHTON  Brighton  ...        0.0        0.0      0.0   

  GLVAL_NR CRHOUSPCT MUNGL1PCT AOEGL_HS AOEGL_NR              E911ADDR  \
0    5

In [ ]:

parcel_gdf = parcel_gdf[["SPAN", "PARCID", "TOWN", "TNAME", 
                         "DESCPROP", "CAT", "RESCODE", "ACRESGL", "REAL_FLV",
                         "NRES_FLV", "LAND_LV", "HSITEVAL", "geometry"]]


In [ ]:

flood_gdf = gpd.read_file("/Users/iansargent/Desktop/ORCA/Steamlit App Testing/VT_Flood_Hazard.geojson")
flood_gdf = flood_gdf[["OBJECTID", "FLD_ZONE", "STUDY_TYP", "ZONE_SUBTY", "SFHA_TF", "geometry" ]]

print(flood_gdf.columns)


Index(['OBJECTID', 'FLD_ZONE', 'STUDY_TYP', 'ZONE_SUBTY', 'SFHA_TF',
       'geometry'],
      dtype='object')


In [ ]:

projected_crs = 'EPSG:3857' 

parcels_proj = parcel_gdf.to_crs(projected_crs)
flood_proj = flood_gdf.to_crs(projected_crs)

parcel_flood = gpd.sjoin(parcels_proj, flood_proj, how='inner', predicate='within')


In [ ]:

parcel_flood = parcel_flood.drop_duplicates(subset=['PARCID'])


# Define risk zones
high_risk_zones = ['A', 'AE', 'AH', 'AO', 'V', 'VE']
low_moderate_risk_zones = ['B', 'X']

# Filter flood zones by risk level
high_risk_flood = parcel_flood[parcel_flood['FLD_ZONE'].isin(high_risk_zones)]
low_risk_flood = parcel_flood[parcel_flood['FLD_ZONE'].isin(low_moderate_risk_zones)]

# Calculate summary statistics
def get_zone_stats(df, risk_level):
    return {
        'Risk Level': risk_level,
        'Zone Counts': df['FLD_ZONE'].value_counts().to_dict(),
        'Total Parcels': df['PARCID'].nunique(),
        'Total Value ($)': df['REAL_FLV'].sum(),
        'Avg Value per Parcel ($)': df['REAL_FLV'].mean(),
        '% of All Parcels': f"{(len(df)/len(parcel_flood))*100:.1f}%"
    }

high_risk_stats = get_zone_stats(high_risk_flood, "High Risk")
low_risk_stats = get_zone_stats(low_risk_flood, "Low/Moderate Risk")

# Format and print results
print("\n" + "="*50)
print(" " * 15 + "FLOOD RISK ANALYSIS REPORT")
print("="*50 + "\n")

def print_stats(stats):
    print(f"\n{stats['Risk Level']} Zones:")
    print("-"*40)
    print("Zone Type Counts:")
    for zone, count in stats['Zone Counts'].items():
        print(f"  {zone}: {count:,}")
    
    print(f"\nTotal Parcels: {stats['Total Parcels']:,}")
    print(f"Total Assessed Value: ${stats['Total Value ($)']:,.2f}")
    print(f"Average Value per Parcel: ${stats['Avg Value per Parcel ($)']:,.2f}")
    print(f"Percentage of All Parcels: {stats['% of All Parcels']}")

print_stats(high_risk_stats)
print_stats(low_risk_stats)

# Print comparison summary
print("\n" + "="*50)
print("SUMMARY COMPARISON")
print("="*50)
print(f"\nHigh Risk parcels represent {high_risk_stats['% of All Parcels']} of total parcels")
print(f"but account for ${high_risk_stats['Total Value ($)']:,.2f} in property value")
print(f"\nAverage parcel value in High Risk zones: ${high_risk_stats['Avg Value per Parcel ($)']:,.2f}")
print(f"Average parcel value in Low/Moderate Risk zones: ${low_risk_stats['Avg Value per Parcel ($)']:,.2f}")
